In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import torch
import torch.nn as nn
import os

# --- 1. Constantes e Configuração ---
class Config:
    TEMP_AMBIENTE = 25.0        # Temperatura ambiente em ºC
    R_VERDADEIRO = 0.005      # Taxa de resfriamento real (1/s)
    T0 = 90.0           # Temperatura inicial do café (ºC)
    INTERVALO_TEMPO = (0, 200)   # Intervalo de tempo para solução numérica
    PONTOS_AVALIACAO = 200 # Número de pontos para avaliar a solução numérica/analítica
    NUM_PONTOS_SINTETICOS = 10 # Manter baixo
    DESVIO_PADRAO_RUIDO = 0.5 # Desvio padrão do ruído para dados sintéticos
    TEMPO_EXTRAPOLACAO = 1000 # Tempo máximo para gráficos de extrapolação
    PONTOS_EXTRAPOLACAO = 500 # Número de pontos para extrapolação
    EPOCAS_NN_SIMPLES = 40000    # Épocas para treinamento da NN simples
    LR_NN_SIMPLES = 1e-4        # Taxa de aprendizado para NN simples
    
    # --- AJUSTES PARA MELHORAR EXTRAPOLAÇÃO DA PINN COM POUCOS DADOS ---
    EPOCAS_PINN_FASE1 = 20000 # Reduzido para "warm-up" rápido com dados
    EPOCAS_PINN_FASE2 = 120000 # Aumentado para mais treinamento com física em todo o domínio
    LR_PINN_FASE1 = 5e-5 
    LR_PINN_FASE2 = 5e-5 
    PESO_PERDA_FISICA = 50000 # AUMENTADO SIGNIFICATIVAMENTE
    CHUTE_INICIAL_R = 0.01 # Chute inicial para 'r' na PINN
    FATOR_NORMALIZACAO = 200 # Fator para normalizar o tempo para a PINN

# Define sementes aleatórias para reprodutibilidade
np.random.seed(42)
torch.manual_seed(42)

# --- 2. Funções Principais (ODE, Solução Analítica, Plotagem) ---

def equacao_diferencial_resfriamento(t, T, r, T_ambiente):
    """Define a equação diferencial ordinária para o resfriamento."""
    return r * (T_ambiente - T)

def solucao_analitica(t, T_ambiente, T0, r):
    """Calcula a solução analítica para o problema de resfriamento."""
    return T_ambiente + (T0 - T_ambiente) * np.exp(-r * t)

def plotar_curvas_temperatura(dados_x, lista_dados_y, rotulos, titulo, rotulo_x='Tempo (s)', rotulo_y='Temperatura (°C)', dados_dispersao=None):
    """
    Plota múltiplas curvas de temperatura em um único gráfico.
    Opcionalmente plota dados de dispersão.
    """
    plt.figure(figsize=(10, 6))
    estilos_linha = ['--', '-', ':', '-.']
    for i, (dados_y, rotulo) in enumerate(zip(lista_dados_y, rotulos)):
        plt.plot(dados_x, dados_y, label=rotulo, linestyle=estilos_linha[i % len(estilos_linha)])
    
    if dados_dispersao is not None:
        plt.scatter(dados_dispersao[0], dados_dispersao[1], color='red', label='Dados Sintéticos (com ruído)', zorder=5)

    plt.xlabel(rotulo_x)
    plt.ylabel(rotulo_y)
    plt.title(titulo)
    plt.legend()
    plt.grid(True)
    plt.show()

# --- 3. Modelos de Rede Neural ---

class RegressorSimples(nn.Module):
    """Uma rede neural feedforward simples para regressão."""
    def __init__(self):
        super().__init__()
        self.rede = nn.Sequential(
            nn.Linear(1, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
        )
            
    def forward(self, x):
        return self.rede(x)

class PINNAprenderR(nn.Module):
    """
    Rede Neural Informada por Física (PINN) para aprender a taxa de resfriamento 'r'.
    Normaliza a entrada de tempo internamente para melhor desempenho.
    """
    def __init__(self, fator_normalizacao):
        super().__init__()
        self.fator_normalizacao = fator_normalizacao
        self.rede = nn.Sequential(
            nn.Linear(1, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
        )
        # Inicializa log_r para evitar 'r' negativo e permitir aprendê-lo
        self.log_r = nn.Parameter(torch.log(torch.tensor(Config.CHUTE_INICIAL_R)))

    def forward(self, t_normalizado):
        return self.rede(t_normalizado)

    def obter_r(self):
        """Retorna a taxa de resfriamento 'r' aprendida."""
        return torch.exp(self.log_r)

# --- 4. Funções de Treinamento ---

def treinar_nn_simples(tensor_t_treino, tensor_T_treino, tensor_t_extrapolacao, funcao_solucao_analitica):
    """Treina e avalia uma rede neural de regressão simples."""
    print("\n--- Treinando Rede Neural Simples ---")
    modelo = RegressorSimples()
    otimizador = torch.optim.Adam(modelo.parameters(), lr=Config.LR_NN_SIMPLES)
    funcao_perda = nn.MSELoss()

    for epoca in range(Config.EPOCAS_NN_SIMPLES):
        modelo.train()
        otimizador.zero_grad()
        saida = modelo(tensor_t_treino)
        perda = funcao_perda(saida, tensor_T_treino)
        perda.backward()
        otimizador.step()
        if epoca % 500 == 0:
            print(f"Época {epoca}: Perda = {perda.item():.6f}")

    modelo.eval()
    with torch.no_grad():
        T_predito_nn = modelo(tensor_t_extrapolacao).numpy()
        T_verdadeiro_extrap = funcao_solucao_analitica(tensor_t_extrapolacao.numpy())

    return T_predito_nn, T_verdadeiro_extrap

def treinar_pinn(tensor_t_treino, tensor_T_treino, tensor_t_extrapolacao, funcao_solucao_analitica, fator_normalizacao, T_ambiente):
    """Treina uma Rede Neural Informada por Física (PINN) para aprender 'r'."""
    print("\n--- Treinando PINN para Aprender 'r' ---")
    modelo = PINNAprenderR(fator_normalizacao)
    funcao_perda = nn.MSELoss()

    # --- ALTERAÇÃO AQUI: Define pontos físicos (normalizados) cobrindo todo o intervalo de EXTRAPOLAÇÃO ---
    # Isso é CRUCIAL para que a física guie a rede além dos pontos de dados.
    max_t_fisico = Config.TEMPO_EXTRAPOLACAO 
    t_fisico_normalizado = torch.linspace(0, max_t_fisico / fator_normalizacao, 
                                          Config.PONTOS_EXTRAPOLACAO * 2).view(-1, 1).requires_grad_(True)
    # Usei * 2 para ter mais pontos de colocação do que pontos de avaliação, ajudando na imposição da física.
    
    # Fase 1: Treinamento somente com dados
    otimizador_fase1 = torch.optim.Adam(modelo.parameters(), lr=Config.LR_PINN_FASE1)
    print("\n--- Fase 1: Treinamento somente com Dados ---")
    for epoca in range(Config.EPOCAS_PINN_FASE1):
        otimizador_fase1.zero_grad()
        T_predito_dados = modelo(tensor_t_treino)
        perda = funcao_perda(T_predito_dados, tensor_T_treino)
        perda.backward()
        otimizador_fase1.step()
        if epoca % 500 == 0:
            valor_r = modelo.obter_r().item()
            print(f"Época {epoca}: Perda = {perda.item():.6f}, r (estimado) = {valor_r:.6f}")

    # Fase 2: Treinamento com Dados + Física
    otimizador_fase2 = torch.optim.Adam(modelo.parameters(), lr=Config.LR_PINN_FASE2)
    print("\n--- Fase 2: Treinamento com Dados + Física ---")
    for epoca in range(Config.EPOCAS_PINN_FASE2):
        otimizador_fase2.zero_grad()
        T_predito_dados = modelo(tensor_t_treino)
        perda_dados = funcao_perda(T_predito_dados, tensor_T_treino)

        # Cálculo da perda de física
        T_predito_fisico = modelo(t_fisico_normalizado)
        # Regra da cadeia para a derivada: dT/dt = (dT/dt_norm) * (dt_norm/dt) = (dT/dt_norm) * (1/fator_normalizacao)
        dTdt_normalizado = torch.autograd.grad(T_predito_fisico, t_fisico_normalizado, grad_outputs=torch.ones_like(T_predito_fisico),
                                                retain_graph=True, create_graph=True)[0]
        dTdt = dTdt_normalizado / fator_normalizacao

        r = modelo.obter_r()
        # Reorganiza a EDO: dT/dt - r*(T_ambiente - T) = 0 => dT/dt + r*(T - T_ambiente) = 0
        f = dTdt + r * (T_predito_fisico - T_ambiente)
        perda_fisica = torch.mean(f**2)

        perda_total = perda_dados + Config.PESO_PERDA_FISICA * perda_fisica
        perda_total.backward()
        otimizador_fase2.step()

        if epoca % 500 == 0:
            valor_r = modelo.obter_r().item()
            print(f"Época {epoca}: Perda Total = {perda_total.item():.6f}, Perda Dados = {perda_dados.item():.6f}, "
                  f"Perda Física = {perda_fisica.item():.6f}, r (estimado) = {valor_r:.6f}")

    r_aprendido = modelo.obter_r().item()
    with torch.no_grad():
        T_predito_pinn = modelo(tensor_t_extrapolacao).numpy()

    return T_predito_pinn, r_aprendido

# --- 5. Bloco de Execução Principal ---

def main():
    # --- Parte 1: Solução Analítica e Numérica (RK45) ---
    print("--- Soluções Analítica e Numérica (RK45) ---")
    
    # Solução da EDO usando solve_ivp
    sol = solve_ivp(
        lambda t, T: equacao_diferencial_resfriamento(t, T, Config.R_VERDADEIRO, Config.TEMP_AMBIENTE),
        Config.INTERVALO_TEMPO,
        [Config.T0],
        t_eval=np.linspace(Config.INTERVALO_TEMPO[0], Config.INTERVALO_TEMPO[1], Config.PONTOS_AVALIACAO),
        method='RK45'
    )
    T_numerico = sol.y[0]
    t_avaliacao = sol.t
    T_analitico = solucao_analitica(t_avaliacao, Config.TEMP_AMBIENTE, Config.T0, Config.R_VERDADEIRO)

    plotar_curvas_temperatura(
        t_avaliacao,
        [T_analitico, T_numerico],
        ['Solução Analítica', 'Solução Numérica (RK45)'],
        'Resfriamento de uma Caneca de Café'
    )

    # --- Parte 2: Geração e Visualização de Dados Sintéticos ---
    print("\n--- Geração e Visualização de Dados Sintéticos ---")
    t_treino = np.linspace(Config.INTERVALO_TEMPO[0], Config.INTERVALO_TEMPO[1], Config.NUM_PONTOS_SINTETICOS)
    T_limpo = solucao_analitica(t_treino, Config.TEMP_AMBIENTE, Config.T0, Config.R_VERDADEIRO)
    ruido = np.random.normal(0, Config.DESVIO_PADRAO_RUIDO, size=T_limpo.shape)
    T_com_ruido = T_limpo + ruido

    plotar_curvas_temperatura(
        t_avaliacao,
        [solucao_analitica(t_avaliacao, Config.TEMP_AMBIENTE, Config.T0, Config.R_VERDADEIRO)],
        ['Solução Analítica'],
        'Dados Sintéticos Gerados a Partir da Solução Analítica',
        dados_dispersao=(t_treino, T_com_ruido)
    )

    # Prepara dados para as Redes Neurais
    t_extrapolacao = np.linspace(0, Config.TEMPO_EXTRAPOLACAO, Config.PONTOS_EXTRAPOLACAO)
    
    # Para NN Simples
    tensor_t_treino_nn = torch.tensor(t_treino, dtype=torch.float32).view(-1, 1)
    tensor_T_treino_nn = torch.tensor(T_com_ruido, dtype=torch.float32).view(-1, 1)
    tensor_t_extrapolacao_nn = torch.tensor(t_extrapolacao, dtype=torch.float32).view(-1, 1)

    # Para PINN (tempo normalizado)
    t_treino_norm = t_treino / Config.FATOR_NORMALIZACAO
    t_extrapolacao_norm = t_extrapolacao / Config.FATOR_NORMALIZACAO
    tensor_t_treino_pinn = torch.tensor(t_treino_norm, dtype=torch.float32).view(-1, 1)
    tensor_T_treino_pinn = torch.tensor(T_com_ruido, dtype=torch.float32).view(-1, 1)
    tensor_t_extrapolacao_pinn = torch.tensor(t_extrapolacao_norm, dtype=torch.float32).view(-1, 1)

    # --- Parte 3: Regressão com Rede Neural Simples ---
    T_predito_nn, T_verdadeiro_extrap_nn = treinar_nn_simples(
        tensor_t_treino_nn, tensor_T_treino_nn, tensor_t_extrapolacao_nn, 
        lambda t: solucao_analitica(t, Config.TEMP_AMBIENTE, Config.T0, Config.R_VERDADEIRO)
    )
    plotar_curvas_temperatura(
        t_extrapolacao,
        [T_verdadeiro_extrap_nn, T_predito_nn],
        ['Solução Analítica', 'NN de Regressão Simples'],
        'Regressão Simples vs. Solução Analítica (Extrapolação até 1000s)',
        dados_dispersao=(t_treino, T_com_ruido)
    )

    # --- Parte 4: Rede Neural Informada por Física (PINN) ---
    T_predito_pinn, r_aprendido = treinar_pinn(
        tensor_t_treino_pinn, tensor_T_treino_pinn, tensor_t_extrapolacao_pinn, 
        lambda t: solucao_analitica(t, Config.TEMP_AMBIENTE, Config.T0, Config.R_VERDADEIRO),
        Config.FATOR_NORMALIZACAO, Config.TEMP_AMBIENTE
    )

    T_verdadeiro_extrap_pinn = solucao_analitica(t_extrapolacao, Config.TEMP_AMBIENTE, Config.T0, Config.R_VERDADEIRO)
    plotar_curvas_temperatura(
        t_extrapolacao,
        [T_verdadeiro_extrap_pinn, T_predito_pinn],
        ['Solução Analítica', f'PINN (r aprendido = {r_aprendido:.5f})'],
        'PINN aprendendo r automaticamente (Extrapolação até 1000s)',
        dados_dispersao=(t_treino, T_com_ruido)
    )

if __name__ == "__main__":
    main()